In [124]:
using BenchmarkTools

In [125]:
import Collective
import PuzzleTools

In [198]:
words = lowercase.(strip.(split(readstring("data/113809of.fic"))));

In [275]:
reload("Collective")

INFO: Recompiling stale cache file /Users/rdeits/.julia/lib/v0.5/Collective.ji for module Collective.


In [276]:
corpus = Collective.Corpus(words[1:Int(round(length(words) / 10000)):end]);

In [277]:
rubik = 

8-element Array{String,1}:
 "armoredrecon"     
 "hypapante"        
 "commemorativebats"
 "derricktruck"     
 "brownrot"         
 "attorneysgeneral" 
 "sacrosanct"       
 "impromptu"        

In [278]:
sort([f for f in Collective.analyze(corpus, rubik) if true])

681-element Array{Collective.FeatureResult,1}:
 Collective.FeatureResult("contains at least 2 repeated letters",Collective.#39,Bool[true,true,true,true,true,true,true,true],0.000138185)             
 Collective.FeatureResult("contains 't' at index 8",Collective.#30,Bool[false,true,false,true,true,false,false,true],0.000259201)                      
 Collective.FeatureResult("has 8 reverse alphabetical bigrams",Collective.#49,Bool[false,false,true,false,false,true,false,false],0.000316072)         
 Collective.FeatureResult("contains at least 3 repeated letters",Collective.#39,Bool[true,false,true,true,false,true,true,false],0.000350198)          
 Collective.FeatureResult("has at least 8 reverse alphabetical bigrams",Collective.#50,Bool[false,false,true,false,false,true,false,false],0.000391839)
 Collective.FeatureResult("contains 5 repeated letters",Collective.#38,Bool[false,false,true,false,false,true,false,false],0.000520469)                
 Collective.FeatureResult("contains at le

In [263]:
sort(Collective.analyze(corpus, ["citygates", "impulsive", "clickspam", "baptistry", "leviathan", "policecar", "coupdetat", "sforzando", "cartwheel"]))[1:5]

5-element Array{Collective.FeatureResult,1}:
 Collective.FeatureResult("has 8 unique letters",Collective.#34,Bool[true,true,true,true,true,true,true,true,true],5.90962e-8)                  
 Collective.FeatureResult("contains 'a' at index 8",Collective.#30,Bool[false,false,true,false,true,true,true,false,false],4.74683e-5)          
 Collective.FeatureResult("contains 1 repeated letters",Collective.#38,Bool[true,true,true,true,true,true,true,true,true],0.000215179)          
 Collective.FeatureResult("contains 'a' at index 2 from end",Collective.#31,Bool[false,false,true,false,true,true,true,false,false],0.000933254)
 Collective.FeatureResult("has 5 alphabetical bigrams",Collective.#44,Bool[true,true,false,true,false,false,true,true,false],0.00184615)        

In [264]:
sort(Collective.analyze(corpus, ["levitate", "inanimate", "sizes", "lightly", "crocodile", "legislator", "carousels"]))[1:5]

5-element Array{Collective.FeatureResult,1}:
 Collective.FeatureResult("contains 'l' at index 1",Collective.#30,Bool[true,false,false,true,false,true,false],0.00110508)        
 Collective.FeatureResult("has scrabble score 11",Collective.#28,Bool[true,true,false,false,false,true,true],0.00313566)           
 Collective.FeatureResult("contains 'l' at index 2 from end",Collective.#31,Bool[false,false,false,true,true,false,true],0.0095657)
 Collective.FeatureResult("contains 'l' at index 8",Collective.#30,Bool[false,false,false,false,true,false,true],0.00987603)       
 Collective.FeatureResult("has 4 reverse alphabetical bigrams",Collective.#46,Bool[true,true,false,false,true,true,true],0.011867) 

In [122]:
sort(Collective.analyze(corpus, ["thumbtacks", "monologue", "testimony", "camel", "meteorology", "trampoline", "achievement"]))[1:5]

5-element Array{Collective.FeatureResult,1}:
 Collective.FeatureResult("contains m",Bool[true,true,true,true,true,true,true],0.000187263)                        
 Collective.FeatureResult("contains t at index 1",Bool[true,false,true,false,false,true,false],0.00981769)          
 Collective.FeatureResult("contains h at index 9 from end",Bool[true,false,false,false,false,false,true],0.0184008) 
 Collective.FeatureResult("contains n at index 2 from end",Bool[false,false,true,false,false,true,true],0.0252534)  
 Collective.FeatureResult("contains t at index 10 from end",Bool[true,false,false,false,false,true,false],0.0285695)

In [123]:
word = "hello"
@benchmark $fset.evaluate($word)

BenchmarkTools.Trial: 
  memory estimate:  3.30 kb
  allocs estimate:  44
  --------------
  minimum time:     15.567 μs (0.00% GC)
  median time:      16.500 μs (0.00% GC)
  mean time:        19.378 μs (2.30% GC)
  maximum time:     4.539 ms (98.04% GC)
  --------------
  samples:          10000
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%